In [1]:
import pandas as pd

import csv
import time
import urllib.request

from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
# 検索して採捕のページのURLを入れます。
haiku_url = "http://www.haiku-data.jp/search.php?next_i=0&search_type=and&kigo=%E6%98%A5&database=%E7%8F%BE%E4%BB%A3%E4%BF%B3%E5%8F%A5%E5%8D%94%E4%BC%9A%E3%80%8C%E7%8F%BE%E4%BB%A3%E4%BF%B3%E5%8F%A5%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E3%80%8D&order=work_up#result"

In [3]:
# データフレームの定義
df = pd.Series()

In [4]:
def writedf():
    html = urllib.request.urlopen(haiku_url)
    soup = BeautifulSoup(html, 'lxml')
    
    # aタグの要素を全て取得する
    haikus = soup.find_all("a") 

    for haiku in haikus:
        global df
        
        # タグの中身を取得
        haiku = haiku.text
        
        # タグの中身が空以外のものを代入
        if haiku != "":
            new = pd.Series(haiku)
            df = df.append(new, ignore_index=True)

In [5]:
def movepage():
    global haiku_url
    
    # PhantomJSのドライバを得る
    browser = webdriver.PhantomJS()
    
    # 暗黙的な待機を最大3秒。ドライバの初期化待ち
    browser.implicitly_wait(3)
    
    # URLの読み込み
    browser.get(haiku_url)
    
    # 遷移
    browser.find_element_by_partial_link_text('次へ').click()

    # URL取得
    haiku_url = browser.current_url

In [6]:
for i in range(10):
    # writecsv()で次へが見つからなかったらbreak
    try:
        writedf()
        movepage()
    except:
        break

In [7]:
garbage_words = ["次へ "," 前へ","▲検索条件の変更","サイトマップ","お問い合わせ"]

for garbage_word in garbage_words:
    global df
    df = df[df != garbage_word]

data=str(datetime.now())
csvname = data+".csv"
df.to_csv(csvname, index=False)